# Human Value Detection

## Imports

In [2]:
import os

import transformers
%load_ext autoreload
%autoreload 2

from utilities import *
from models.bertOne import BertOne

from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from models.randomUniformClassifier import RandomUniformClassifier
from models.majorityCalssifier import MajorityClassifier

from transformers import AutoTokenizer

from drTorch.callbacks import EarlyStopper

from drTorch.metrics import F1_Score
from drTorch.metrics import F1_Score_Multi_Labels
from drTorch.utilities import *
from drTorch.wrappers import OptimizerWrapper
from drTorch.wrappers import Criterion

import numpy as np
import torch
import pandas as pd


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print('Device: %s' % device)


Device: cuda


## Defining constants and flags

In [4]:
# PATHS AND DATAFRAME CREATION
DATA_DIR = "data"
ARGUMENTS_DIR = os.path.join(DATA_DIR, "arguments")
LABELS_DIR = os.path.join(DATA_DIR, "labels")

BERT_MODELS_DIRECTORY = "bert_models"
BERT_VERSIONS=["bert-base-uncased"]

# CONSTANTS 
N_LABELS = 4
N_CLASSES = 2
BATCH_SIZE = 1
#os.environ["TOKENIZERS_PARALLELISM"] = "true"

CLASS_2_ONE_HOT = {class_label: np.eye(N_CLASSES)[i].astype(float).tolist() for i, class_label in enumerate(range(N_CLASSES))}


## Task 1

### Visualizing the data

In [5]:
# convert files in dataframes
train_arg_df, val_arg_df, test_arg_df = create_dfs(ARGUMENTS_DIR)
train_labels_df, val_labels_df, test_labels_df = create_dfs(LABELS_DIR)

print("Let's visualize the data: ")
display(train_arg_df.head(5))
display(train_labels_df.head(5))


Let's visualize the data: 


,Conclusion,Stance,Premise
Argument ID,,,
A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...
A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...
A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...
A01007,We should abolish capital punishment,against,capital punishment is sometimes the only optio...
A01008,We should ban factory farming,against,factory farming allows for the production of c...


,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,Security: societal,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
Argument ID,,,,,,,,,,,,,,,,,,,,
A01002,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
A01005,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
A01006,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
A01007,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0
A01008,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0


### Mapping labels to level-3 categories

In [6]:
mapping = define_mapping()
train_labels_df, val_labels_df, test_labels_df = map_to_level_3(mapping, train_labels_df, val_labels_df, test_labels_df) 

print("The training labels after the mapping are the following: ")
train_labels_df


The training labels after the mapping are the following: 


,Openess_to_change,Self_enhancement,Conservation,Self_transcendence
Argument ID,,,,
A01002,0,0,1,0
A01005,0,0,1,0
A01006,0,1,1,0
A01007,0,0,1,0
A01008,0,0,1,1
...,...,...,...,...
E08016,0,1,1,1
E08017,0,0,1,1
E08018,0,0,0,1


### One-hot encoding, tokenization and data loaders building

In [7]:
if os.path.exists(BERT_MODELS_DIRECTORY):
    bert_versions_paths = os.listdir(BERT_MODELS_DIRECTORY)
else:    
    bert_versions_paths = download_bert_models(BERT_MODELS_DIRECTORY, BERT_VERSIONS)  
    

In [8]:
model_inputs = {"C":["Conclusion"], "CP":["Conclusion", "Premise"], "CPS":["Conclusion", "Premise", "Stance"]}

dataloader_train_C, dataloader_val_C, dataloader_test_C  = build_dataloaders(train_df=train_arg_df, 
                                                                             val_df=val_arg_df, 
                                                                             test_df=test_arg_df, 
                                                                             train_labels_df=train_labels_df, 
                                                                             val_labels_df=val_labels_df, 
                                                                             test_labels_df=test_labels_df, 
                                                                             one_hot_mapping=CLASS_2_ONE_HOT, 
                                                                             bert_version="bert-base-uncased", 
                                                                             model_input=model_inputs["C"],
                                                                             custom_dataset_builder=CustomDataset_C,
                                                                             batch_size=BATCH_SIZE, 
                                                                             shuffle=True)

dataloader_train_CP, dataloader_val_CP, dataloader_test_CP  = build_dataloaders(train_df=train_arg_df, 
                                                                                val_df=val_arg_df, 
                                                                                test_df=test_arg_df, 
                                                                                train_labels_df=train_labels_df, 
                                                                                val_labels_df=val_labels_df, 
                                                                                test_labels_df=test_labels_df, 
                                                                                one_hot_mapping=CLASS_2_ONE_HOT, 
                                                                                bert_version="bert-base-uncased", 
                                                                                model_input=model_inputs["CP"],
                                                                                custom_dataset_builder=CustomDataset_CP,
                                                                                batch_size=BATCH_SIZE, 
                                                                                shuffle=True)

dataloader_train_CPS, dataloader_val_CPS, dataloader_test_CPS  = build_dataloaders(train_df=train_arg_df, 
                                                                                   val_df=val_arg_df, 
                                                                                   test_df=test_arg_df, 
                                                                                   train_labels_df=train_labels_df, 
                                                                                   val_labels_df=val_labels_df, 
                                                                                   test_labels_df=test_labels_df, 
                                                                                   one_hot_mapping=CLASS_2_ONE_HOT, 
                                                                                   bert_version="bert-base-uncased", 
                                                                                   model_input=model_inputs["CPS"],
                                                                                   custom_dataset_builder=CustomDataset_CPS,
                                                                                   batch_size=BATCH_SIZE, 
                                                                                   shuffle=True)


## Task 2 

### Models Definition

####  1) Random uniform classifier

In [23]:
# Create an instance of the random uniform classifier
random_classifier = RandomUniformClassifier(N_LABELS)

# Make predictions on the test set
predicted_labels = random_classifier.predict(test_arg_df)

# Accuracy of the Random Classifier
accuracy = accuracy_score(test_labels_df, predicted_labels)
print(f'Accuracy of the model over all the classes: {accuracy}\n')

# F1 scores on the different labels singularly taken 
f1_scorer = F1_Score_Multi_Labels(name='F1_macro_avg', num_labels=N_LABELS, num_classes=N_CLASSES, compute_mean=False)
f1_scores_random_classifier = f1_scorer(torch.tensor(test_labels_df.values.tolist()), torch.tensor(predicted_labels))
# AVG f1 score
f1_avg_random_classifier = np.average(f1_scores_random_classifier)


for idx, f1_score in enumerate(f1_scores_random_classifier):
    print(f" - {train_labels_df.columns[idx]} F1: {f1_score}")

print(f'\nAverage F1: {f1_avg_random_classifier}')

Accuracy of the model over all the classes: 0.06725888324873096

 - Openess_to_change F1: 0.4709263459646603
 - Self_enhancement F1: 0.4953258965986781
 - Conservation F1: 0.4814643653680002
 - Self_transcendence F1: 0.5120320656451882

Average F1: 0.4899371683941317


####  1) Majority classifier

In [29]:
majority_classifier = MajorityClassifier()

# Train the majority classifier (even though in practice, no training is needed)
majority_classifier.fit(train_labels_df)

# Make predictions on the test set
predicted_labels = majority_classifier.predict(test_labels_df)

# Accuracy of the Majority Classifier
accuracy = accuracy_score(test_labels_df, predicted_labels)
print(f'Accuracy of the model over all the classes: {accuracy}\n')

# F1 scores on the different labels singularly taken 
f1_scorer = F1_Score_Multi_Labels(name='F1_macro_avg', num_labels=N_LABELS, num_classes=N_CLASSES, compute_mean=False)
f1_scores_majority_classifier = f1_scorer(torch.tensor(test_labels_df.values.tolist()), torch.tensor(predicted_labels))
# AVG F1 score
f1_avg_majority_classifier = np.average(f1_scores_majority_classifier)


for idx, f1_score in enumerate(f1_scores_majority_classifier):
    print(f" - {train_labels_df.columns[idx]} F1: {f1_score}")

print(f'\nAverage F1: {f1_avg_majority_classifier}')


Accuracy of the model over all the classes: 0.13642131979695432

 - Openess_to_change F1: 0.41150112023898433
 - Self_enhancement F1: 0.3708582834331337
 - Conservation F1: 0.4152133580705009
 - Self_transcendence F1: 0.32850447379633574

Average F1: 0.3815193088847386


####  3) Bert

In [2]:
optimizer_test = OptimizerWrapper(torch.optim.Adam, identifier=f'lr={1e-5}', optimizer_partial_params={'lr': 1e-6})          
criterion_test = Criterion('loss', loss_function=torch.nn.BCEWithLogitsLoss(reduction='none'), reduction_function=torch.sum)

bert1 = BertOne(dropout_prob=0.3, hidden_size= 768, bert_version='bert-base-uncased').to(device)

bert1_history = bert1.fit(train_loader=dataloader_train_C, 
                          val_loader=dataloader_val_C, 
                          criterion=criterion_test, 
                          metrics=[F1_Score_Multi_Labels('F1_macro_avg', num_labels=N_LABELS, num_classes=N_CLASSES)], 
                          optimizer=optimizer_test,
                          # early_stopper=EarlyStopper(monitor='F1_macro', patience=4, delta=0, mode='max', restore_weights=True),
                          num_epochs=200)

"""
loss homework 1 con batch size 1
[0.4, 0.5, 0.8, ... 0.2] 46 loss

loss homework 2 con batch size 1
        [[0.4339, 0.4490],
        [0.5538, 0.6676],
        [0.4004, 0.3562],
        [0.6964, 0.6620]]
"""

In [13]:
### Test sul modello

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
config=transformers.BertConfig.from_pretrained('bert-base-uncased')
text = "Replace me by any text you'd like pollo."
encoded_input = tokenizer(text, return_tensors='pt')

model = transformers.BertModel.from_pretrained("bert-base-uncased")
output = model(**encoded_input)
print(model.config)
"""p_d = BertOne()
output1 = p_d(**encoded_input)"""


BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}


'p_d = BertOne()\noutput1 = p_d(**encoded_input)'

In [3]:
from torch.nn.functional import max_pool2d

print(output[0].shape)
output_2 = max_pool2d(output[0], kernel_size=(14,1))

output_2[:,0,:].shape

NameError: name 'OptimizerWrapper' is not defined